In [1]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize()

# Create an interactive map
Map = geemap.Map(center=[55.0, 10.0], zoom=6)
Map.add_basemap("hybrid")
print("Select a point location on the map using the 'Draw a marker' tool, then run the next cell.")
Map  # Displays the interactive map in Jupyter Notebook

Select a point location on the map using the 'Draw a marker' tool, then run the next cell.


Map(center=[55.0, 10.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [3]:
import ee

# Ensure Map is defined
if 'Map' not in globals():
    print("Error: The map is not defined. Please run the first cell to create the map.")
else:
    # Get the selected point
    point = Map.draw_last_feature

    if point:
        coords = point.geometry().coordinates().getInfo()
        lon, lat = coords[0], coords[1]
        print(f"Selected Location: Longitude = {lon}, Latitude = {lat}")

        # Function to display the SMAP image and vectorize the selected pixel boundary
        def display_smap_pixel(lon, lat):
            feature = ee.Geometry.Point([lon, lat])

            # Get the latest available SMAP image at this location
            smap_image = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                .filterDate('2019-01-01', '2019-12-31') \
                .filterBounds(feature) \
                .sort('system:time_start', False) \
                .first()  # Get the most recent SMAP image

            if smap_image:
                # Select the 'sm_surface' band
                sm_surface = smap_image.select('sm_surface')

                # Get the projection and scale dynamically
                projection = sm_surface.projection()
                scale = projection.nominalScale().getInfo()  # Resolution in meters

                if scale is None:
                    print("Error: Could not determine SMAP pixel scale.")
                    return

                print(f"SMAP Pixel Scale: {scale} meters")

                # Extract the pixel value at the selected point
                pixel_value = sm_surface.reduceRegion(
                    reducer=ee.Reducer.first(),
                    geometry=feature,
                    scale=scale,  # Use extracted scale
                    bestEffort=True
                ).get('sm_surface')

                # Ensure the pixel value exists before proceeding
                if pixel_value is None:
                    print("No valid SMAP pixel found at this location.")
                    return

                # Convert the floating-point pixel value to an integer
                scaling_factor = 10000  # To preserve decimal precision
                pixel_value_int = ee.Number(pixel_value).multiply(scaling_factor).int()

                # Convert the extracted pixel value into an EE image
                pixel_value_img = ee.Image.constant(pixel_value_int)

                # Convert SMAP values to integers for vectorization
                sm_surface_int = sm_surface.multiply(scaling_factor).int()

                # Create a mask to isolate the selected pixel
                pixel_mask = sm_surface_int.eq(pixel_value_img)
                isolated_pixel = sm_surface_int.updateMask(pixel_mask)

                # Crop the image to a small buffer around the selected point
                buffer_size = scale * 2  # Buffer size twice the pixel size
                clipped_image = isolated_pixel.clip(feature.buffer(buffer_size))

                # Convert the isolated pixel into a vector using `.reduceToVectors()`
                pixel_vector = clipped_image.reduceToVectors(
                    geometryType='polygon',
                    reducer=ee.Reducer.countEvery(),
                    scale=scale,  # Use extracted scale
                    geometry=clipped_image.geometry(),  # Ensure the operation is bounded
                    bestEffort=True
                )

                # Add the SMAP image to the map
                smap_vis = {
                    'min': 0.0, 
                    'max': 0.5, 
                    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
                }
                Map.addLayer(sm_surface, smap_vis, 'SMAP Soil Moisture')

                # Add the vectorized SMAP pixel boundary in bright yellow
                Map.addLayer(pixel_vector, {'color': 'yellow', 'width': 2}, 'Vectorized SMAP Pixel')

                # Center the map on the selected pixel
                Map.set_center(lon, lat, 9)
                print("SMAP pixel vectorized and displayed.")

            else:
                print("No SMAP data available at the selected location.")

        # Display the SMAP image and vectorized pixel boundary
        display_smap_pixel(lon, lat)

        # Refresh map
        Map

    else:
        print("No point selected. Please click on the map and run this cell again.")


Selected Location: Longitude = -122.975121, Latitude = 49.122661
SMAP Pixel Scale: 10593.328944046672 meters
SMAP pixel vectorized and displayed.


In [4]:
import pandas as pd
from tqdm import tqdm

# Ensure Map is defined
if 'Map' not in globals():
    print("Error: The map is not defined. Please run the first cell to create the map.")
else:
    # Get the selected point again
    point = Map.draw_last_feature

    if point:
        coords = point.geometry().coordinates().getInfo()
        lon, lat = coords[0], coords[1]
        print(f"Extracting SMAP data for Longitude = {lon}, Latitude = {lat}")

        # Function to extract SMAP monthly mean soil moisture data
        def get_smap_monthly_data(lon, lat):
            feature = ee.Geometry.Point([lon, lat])
            smap_list = []

            # Retrieve the scale dynamically from the SMAP image
            smap_sample_image = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
                .filterBounds(feature) \
                .first()  # Get a sample SMAP image
            
            if smap_sample_image:
                scale = smap_sample_image.select('sm_surface').projection().nominalScale().getInfo()
                print(f"Using dynamically extracted SMAP scale: {scale} meters")
            else:
                print("Error: Could not retrieve SMAP scale.")
                return

            for year in tqdm(range(2014, 2020)):  # Extracts data from 2014 to 2019
                for month in range(1, 13):  # Loops over each month
                    start_date = f"{year}-{month:02d}-01"
                    end_date = f"{year}-{month+1:02d}-01" if month < 12 else f"{year+1}-01-01"

                    smSurface = (
                        ee.ImageCollection('NASA/SMAP/SPL4SMGP/007')
                        .filter(ee.Filter.date(start_date, end_date))
                        .filterBounds(feature)
                        .select('sm_surface')
                    )

                    # Calculate monthly mean soil moisture
                    monthly_mean = smSurface.mean().reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=feature,
                        scale=scale,  # Use dynamically extracted scale
                        bestEffort=True
                    ).getInfo()

                    if monthly_mean:
                        df = pd.DataFrame(monthly_mean.items(), columns=['Variable', 'sm_surface'])
                        df['Year'] = year
                        df['Month'] = month
                        df['Longitude'] = lon
                        df['Latitude'] = lat
                        df = df.drop(['Variable'], axis=1)

                        smap_list.append(df)

            if smap_list:
                final_df = pd.concat(smap_list, ignore_index=True)
                final_df.to_csv('smap_monthly_data.csv', mode='a', index=False, header=True)
                print("SMAP data extraction completed. File saved as smap_monthly_data.csv")

        # Extract SMAP monthly mean data
        get_smap_monthly_data(lon, lat)

    else:
        print("No point selected. Please click on the map and run this cell again.")


Extracting SMAP data for Longitude = -122.975121, Latitude = 49.122661
Using dynamically extracted SMAP scale: 10593.328944046672 meters


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [07:53<00:00, 79.00s/it]

SMAP data extraction completed. File saved as smap_monthly_data.csv
